<a href="https://colab.research.google.com/github/joyceee916/ATMS-597-SP-2020/blob/master/Project4_Joyce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd

In [3]:
cd /content/drive/My Drive/ATMS 597/Project4

/content/drive/My Drive/ATMS 597/Project4


Tasks
1. Use multiple linear regression to predict weather using past Global Forecast System (GFS) forecast data. 

2. Use random forest regression to predict weather using past GFS forecast data. 

Predict these variables for the period 6 UTC to 6 UTC the following day: 
* Maximum Temperature (C)
* Minimum Temperature (C)
* Maximum Wind Speed (m/s)
* Total precipitation accumulation (mm)



Day 0 at 12 UTC: you will have a model run (that's given to you)
* can incorporate observations up to but not including 00UTC 
* at the point of 00UTC will predict from 06UTC to 06UTC the next day 

Have 2 observational files (can be used in training, can also be used for verification)
* KCMI_hourly.csv
* KCMI_daily_obs.csv
* ^^ have to sum hourly observations to get total precipitation, has daily values even though the column heading says "hourly"

use pr1h or pr6h to correct for seasonal biases? (seems optional?) 

from the GFS: 
* daily.tar.gz, prof.tar.gz, sfc.tar.gz (UNIX zip files)
* for each run they contain a csv file that has the GFS forecast data 
* sfc.tar.gz - has a lot of data if you want to play around with it (sounds optional)
* prof.tar.gz - some upper air values (includes ~5 pressure levels) (can include if you want to) (sounds optional)
* each run has a timestamp that starts at 12UTC

main challenge: to get data lined up in Pandas 

start simple: start with a few variables and do a multiple linear regression, make sure things are working right 

could generate a persistence variable, try to predict tomorrow's data 

then could slowly add in more variables as you get more confidence 

could select different columns and see if your biases get better or worse as you swap out different columns - could write a function for this? 

2019 as a validation set 

calculate one metric: the RMS error (performance metric)

this week, focus on getting data read in and organized 



NOW: CAN SUBSTITUTE BICYCLE RIDERS IN WITH TEMPERATURE DATA (MODULE 4 NOTEBOOK 1): PREDICT TEMPERATURE TOMORROW 

In [0]:
KCMI_daily_df = pd.read_csv("KCMI_daily.csv", skiprows=8, parse_dates=True, index_col = 'Date')

In [0]:
KCMI_daily_df = KCMI_daily_df.iloc[0:3652, 0:4]

In [43]:
KCMI_daily_df.head()

,Max Hourly Temp (F),Min Hourly Temp (F),Max Wind Speed (mph),Total Precip (in)
Date,,,,
2010-01-01,12,2,15,M
2010-01-02,6,-5,15,0.00
2010-01-03,10,-6,11,0.00
2010-01-04,11,-2,20,0.00
2010-01-05,10,-1,16,M


In [48]:
KCMI_daily_df.rename(columns={'Max Hourly Temp (F)': 'Max_Temp', 'Min Hourly Temp (F)': 'Min_Temp', 'Max Wind Speed (mph)': 'Max_Wind', 'Total Precip (in)': 'Total_Precip'}, inplace = True)
KCMI_daily_df

,Max_Temp,Min_Temp,Max_Wind,Total_Precip
Date,,,,
2010-01-01,12,2,15,M
2010-01-02,6,-5,15,0.00
2010-01-03,10,-6,11,0.00
2010-01-04,11,-2,20,0.00
2010-01-05,10,-1,16,M
...,...,...,...,...
2019-12-27,43,32,15,0.00
2019-12-28,56,32,23,0.23
2019-12-29,59,41,23,0.22


In [0]:
Practice_KCMI_daily = KCMI_daily_df[:]

In [103]:
Practice_KCMI_daily = Practice_KCMI_daily[(Practice_KCMI_daily!='M')]
Practice_KCMI_daily

,Max_Temp,Min_Temp,Max_Wind,Total_Precip
Date,,,,
2010-01-02,6,-5,15,0.00
2010-01-03,10,-6,11,0.00
2010-01-04,11,-2,20,0.00
2010-01-09,9,-9,16,0.00
2010-01-10,19,-12,18,0.00
...,...,...,...,...
2019-12-27,43,32,15,0.00
2019-12-28,56,32,23,0.23
2019-12-29,59,41,23,0.22


In [0]:
# create an array of the dates where Total_Precip=M 
M_dates = KCMI_daily_df.index[(KCMI_daily_df.Total_Precip == 'M')]

Bring In KCMI Hourly Data

In [0]:
KCMI_hourly_df = pd.read_csv("KCMI_hourly.csv", parse_dates=True, index_col = 'Timestamp')

In [45]:
KCMI_hourly_df.head()

,year,month,day,hour,tmpc,dwpc,mslp,wdir,wspd,skct,pr1h,pr6h,doy,woy
Timestamp,,,,,,,,,,,,,,
2005-12-31 19:00:00,2006,1,1,1,0.6,-1.7,1012.6,220.0,2.1,2.0,0.0,NaN,365,52
2005-12-31 20:00:00,2006,1,1,2,0.0,-1.7,1012.9,160.0,2.1,0.0,0.0,NaN,365,52
2005-12-31 21:00:00,2006,1,1,3,-0.6,-2.2,1013.4,170.0,1.5,0.0,0.0,NaN,365,52
2005-12-31 22:00:00,2006,1,1,4,-1.1,-2.2,1012.9,160.0,3.1,0.0,0.0,NaN,365,52
2005-12-31 23:00:00,2006,1,1,5,-1.1,-2.2,1013.1,160.0,3.1,0.0,0.0,NaN,365,52


clean up and rename columns? 

In [0]:
# Use M_dates to select out dates from KCMI_hourly
# Sum hourly values from KCMI_hourly 
# Bring those values back into KCMI_daily 

Linear Regression 

Machine Learning

In [0]:
from sklearn.model_selection import train_test_split

In [127]:
col_mask=Practice_KCMI_daily.isnull().any(axis=0) 
row_mask=Practice_KCMI_daily.isnull().any(axis=1)
Practice_KCMI_daily.loc[row_mask,col_mask]

Practice_KCMI_daily.dropna()

,Max_Temp,Min_Temp,Max_Wind,Total_Precip
Date,,,,
2010-01-02,6,-5,15,0.00
2010-01-03,10,-6,11,0.00
2010-01-04,11,-2,20,0.00
2010-01-09,9,-9,16,0.00
2010-01-10,19,-12,18,0.00
...,...,...,...,...
2019-12-27,43,32,15,0.00
2019-12-28,56,32,23,0.23
2019-12-29,59,41,23,0.22


In [0]:
# TODO change back to KCMI_daily 
x = Practice_KCMI_daily.drop(['Total_Precip'], axis=1)
y = Practice_KCMI_daily['Total_Precip']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=123)

In [130]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1959, 3) (218, 3) (1959,) (218,)


In [0]:
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor 

In [0]:
tree_model = DecisionTreeRegressor()
rf_model = RandomForestRegressor()

In [134]:
tree_model.fit(x_train, y_train) 
### do something about the 'M' values 

ValueError: ignored

In [0]:
## LINEAR REGRESSION 


In [0]:
Daily_Tar = open("daily.tar.gz")